In [1]:
import sudoku


import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
def matrix_to_one_hot(m):
    ret = np.zeros(9*9*9)
    for i, v in enumerate(m.flatten()):
        if v == 0:
            continue
        ret[i*9+v-1] = 1 
    return ret



def one_hot_to_matrix(oh):
    ret = np.zeros(81)
    for i in range(81):
        val = -1
        for v in range(9):
            if oh[i*9+v] != 0:
                val = oh[i*9+v]
            if val != -1:
                ret[i] = v + 1
                break
    return ret

In [3]:
import sudoku
reload(sudoku)


def generate_batch(n):
    puzzles = []

    for _ in range(n):
        result_steps = sudoku.solve(sudoku.create_matrix())
        puzzle = result_steps[1]
        

        if not (sudoku.is_complete(puzzle[-1]) and sudoku.is_correct(puzzle[-1])):
            print 'shit'
        else:
            puzzles.append(puzzle)
            
    puz_x = []
    puz_y = []
    for puzzle in puzzles:
        previous = puzzle[0]
        for step in puzzle[1:-2]:
            if (step > 0).sum() == (previous > 0).sum():
                continue

            puz_x.append(matrix_to_one_hot(previous))

            puz_y.append(matrix_to_one_hot(step - previous))

            previous = step

    #puz_x = np.array(puz_x)
    #puz_y = np.array(puz_y)   
    return puz_x, puz_y

In [4]:
input_size = 729
output_size = 729

In [5]:
import tensorflow as tf

tf.reset_default_graph()

x  = tf.placeholder(tf.float32, shape=[None, input_size])
y_ = tf.placeholder(tf.float32, shape=[None, output_size])

w1 = tf.get_variable("w1", shape=[input_size, 800])
b1 = tf.Variable(tf.zeros([800], dtype=tf.float32))

w3 = tf.Variable(tf.zeros([800, output_size], dtype=tf.float32))
b3 = tf.Variable(tf.zeros([output_size], dtype=tf.float32))

h1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
#h2 = tf.nn.sigmoid(tf.matmul(h1, w2) + b2)
y = tf.matmul(h1, w3) + b3

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y)
train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy)

#with tf.Session() as sess
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [6]:
for i in range(10000):
    x_train, y_train = generate_batch(100)
    train_step.run(feed_dict={x: x_train, y_: y_train})

ResourceExhaustedError: OOM when allocating tensor with shape[1692,729]
	 [[Node: gradients/logistic_loss/Select_1_grad/zeros_like = ZerosLike[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](logistic_loss/Neg)]]

Caused by op u'gradients/logistic_loss/Select_1_grad/zeros_like', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).

In [ ]:
x_test, y_test = generate_batch(100)    
res = y.eval(feed_dict={x: x_train})

In [154]:
res.shape

(4, 729)

In [156]:
for am in res.argmax(axis=1):
    r = 

array([136,  19,  72, 120])

In [108]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, res).ravel()

In [109]:
(tn, fp, fn, tp)

(1532, 2, 91, 375)

In [115]:
reload(sudoku)
puzzles = []

for _ in range(10):
    result_steps = sudoku.solve(sudoku.create_matrix())
    puzzles.append(result_steps)

In [121]:
res = puzzles[0][1]

In [122]:
res[1] - res[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 9, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])